# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_csv = "../output_data/cities.csv"
weather_df = pd.read_csv(weather_csv)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Upernavik,72.7868,-56.1549,26.82,97,99,4.56,GL,1620790040
1,Steamboat Springs,40.4850,-106.8317,37.40,81,40,4.61,US,1620789941
2,Buturlinovka,50.8239,40.6092,46.17,77,94,11.86,RU,1620790041
3,Ternate,0.8000,127.4000,85.98,65,22,2.82,ID,1620790041
4,Busselton,-33.6500,115.3333,63.00,70,94,4.00,AU,1620789756
...,...,...,...,...,...,...,...,...,...
575,Campbellsville,37.3434,-85.3419,55.99,54,75,8.05,US,1620790089
576,Shache,38.4167,77.2406,72.39,24,100,20.69,CN,1620790200
577,Gwadar,25.1216,62.3254,84.47,63,0,8.19,PK,1620790200
578,Mayumba,-3.4320,10.6554,77.16,90,100,3.31,GA,1620790201


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Pull gmaps
gmaps.configure(api_key=g_key)


In [4]:
# Set up cities latatude/longetude and humidity
cities = weather_df[['Lat', 'Lng']]
humidity = weather_df['Humidity']
humidity.max()

100

In [14]:
# Set up heatmap 
humidity_fig = gmaps.figure(center = (20, 0), zoom_level = 2)

# Add heatlayer over map
humidity_heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)

# Print and Save heat map
humidity_fig.add_layer(humidity_heat_layer)
plt.savefig('../output_data/humidity_heatmap')
humidity_fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create ideal weatther conditions Data Frame
high_temp = weather_df['Max Temp'] < 80
low_temp = weather_df['Max Temp'] > 70
wind = weather_df['Wind Speed'] < 10
clouds = weather_df['Cloudiness'] == 0

ideal_weather_df = weather_df.loc[(high_temp) & (low_temp) & (wind) & (clouds)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,Saint-Philippe,-21.3585,55.7679,73.40,73,0,3.44,RE,1620790058
85,Vila Velha,-20.3297,-40.2925,71.01,88,0,8.23,BR,1620789917
132,Livramento,-3.0254,-60.1803,78.80,100,0,2.86,BR,1620790075
147,Saint-Leu,-21.1500,55.2833,73.40,73,0,3.44,RE,1620790080
198,Touros,-5.1989,-35.4608,73.40,94,0,4.61,BR,1620790095
249,Najrān,17.4924,44.1277,77.00,38,0,5.66,SA,1620790110
304,Jalu,29.0331,21.5482,75.72,18,0,8.28,LY,1620790124
342,Bida,9.0833,6.0167,79.57,75,0,4.70,NG,1620790135
359,Nau,40.1525,69.3686,71.60,46,0,6.71,TJ,1620790140
411,Safaga,26.7292,33.9365,77.00,38,0,8.05,EG,1620790155


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create Data Frame for Hotels nearby ideal weather locations
hotel_df = ideal_weather_df.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
66,Saint-Philippe,-21.3585,55.7679,73.40,73,0,3.44,RE,1620790058,
85,Vila Velha,-20.3297,-40.2925,71.01,88,0,8.23,BR,1620789917,
132,Livramento,-3.0254,-60.1803,78.80,100,0,2.86,BR,1620790075,
147,Saint-Leu,-21.1500,55.2833,73.40,73,0,3.44,RE,1620790080,
198,Touros,-5.1989,-35.4608,73.40,94,0,4.61,BR,1620790095,
249,Najrān,17.4924,44.1277,77.00,38,0,5.66,SA,1620790110,
304,Jalu,29.0331,21.5482,75.72,18,0,8.28,LY,1620790124,
342,Bida,9.0833,6.0167,79.57,75,0,4.70,NG,1620790135,
359,Nau,40.1525,69.3686,71.60,46,0,6.71,TJ,1620790140,
411,Safaga,26.7292,33.9365,77.00,38,0,8.05,EG,1620790155,


In [8]:
# Create Url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Geocoordinates
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# Create parameters
params= {
    "keyword" : target_search,
    "radius" : target_radius,
    "type" : target_type,
    "key" : g_key
}


In [9]:
# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Get hotel type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    
    # Add keyword to params dict
    params['location'] = f"{latitude},{longitude}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is: {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    

Closest hotel in Saint-Philippe is: Les Embruns Du Baril.
------------
Closest hotel in Vila Velha is: Hotel Senac Ilha do Boi.
------------
Missing field/result... skipping.
------------
Closest hotel in Saint-Leu is: Iloha Seaview Hotel.
------------
Closest hotel in Touros is: Hotel Vila Gale Touros.
------------
Missing field/result... skipping.
------------
Closest hotel in Jalu is: Jalu Hotel.
------------
Closest hotel in Bida is: House 7 Resort.
------------
Missing field/result... skipping.
------------
Closest hotel in Safaga is: Lamar Resort Abu Soma.
------------
Missing field/result... skipping.
------------
Closest hotel in Manacapuru is: Paraíso D'Ângelo.
------------


In [10]:
# Final Data Frame
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
66,Saint-Philippe,-21.3585,55.7679,73.40,73,0,3.44,RE,1620790058,Les Embruns Du Baril
85,Vila Velha,-20.3297,-40.2925,71.01,88,0,8.23,BR,1620789917,Hotel Senac Ilha do Boi
132,Livramento,-3.0254,-60.1803,78.80,100,0,2.86,BR,1620790075,
147,Saint-Leu,-21.1500,55.2833,73.40,73,0,3.44,RE,1620790080,Iloha Seaview Hotel
198,Touros,-5.1989,-35.4608,73.40,94,0,4.61,BR,1620790095,Hotel Vila Gale Touros
249,Najrān,17.4924,44.1277,77.00,38,0,5.66,SA,1620790110,
304,Jalu,29.0331,21.5482,75.72,18,0,8.28,LY,1620790124,Jalu Hotel
342,Bida,9.0833,6.0167,79.57,75,0,4.70,NG,1620790135,House 7 Resort
359,Nau,40.1525,69.3686,71.60,46,0,6.71,TJ,1620790140,
411,Safaga,26.7292,33.9365,77.00,38,0,8.05,EG,1620790155,Lamar Resort Abu Soma


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations,  info_box_content = hotel_info)

# Display figure
humidity_fig.add_layer(hotel_markers)
plt.savefig('../output_data/hotel_markers')
humidity_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>